In [ ]:
# compare strings
!pip install jellyfish

     |████████████████████████████████| 132 kB 7.9 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73990 sha256=e832ad139a9792e4fddb334baac4f68c0fde839697339be84dfb0b0d31597652
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
import numpy as np
import pandas as pd
import jellyfish
from tqdm import tqdm
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
deep_learning_dir = "/content/gdrive/My Drive/BMI 707 Project"

cui_path = "/data/mapping"

# cui_str is a dataframe with two columns: CUI | concept as string
cui_str = pd.read_csv(deep_learning_dir + cui_path + "/dictionary.csv", sep="|")
str_to_cui = dict(zip(cui_str["STR"], cui_str["CUI"]))

In [ ]:
data_path = "/data_formatting"

train = pd.read_csv(deep_learning_dir + data_path + "/training_data.tsv", sep="\t")
val = pd.read_csv(deep_learning_dir + data_path + "/validation_data.tsv", sep="\t")
test = pd.read_csv(deep_learning_dir + data_path + "/testing_data.tsv", sep="\t")

In [ ]:
def clean_column(col):
  """
  After reading in the files, the disease cells are strings indead of lists
  """
  col = [e.replace("[", '') for e in col]
  col = [e.replace("]", '') for e in col]
  col = [e.replace("'", '') for e in col]
  col = [e.split(", ") for e in col]

  return col


def flatten_lol(l):
  """
  Flatten a list of lists
  """

  return [item for sublist in l for item in sublist]

In [ ]:
diseases_train = clean_column(train["diseases"])
diseases_val = clean_column(val["diseases"])
diseases_test = clean_column(test["diseases"])

In [ ]:
# all diseases in the dataset
diseases = flatten_lol(diseases_train) + flatten_lol(diseases_val) + flatten_lol(diseases_test)

# hand-curation
diseases = list(set(diseases) - set(["type 2", "high-risk", "first-line", "oral", "major", "breast", "immediate", "nontuberculous",
                                     "partial", "waldenstroms", "painful", "inborn", "local", "chronic", "specific", "left", "depression",
                                     "high", "human", "sudden", "mixed", "perennial", "postherpetic", "functional", "prevention", "safety"]))

In [ ]:
def d_to_cuis(disease):
  """
  Find CUI of a given disease. If no CUI matches, split the disease into words and look for CUI's
  of the words. If there is no match even after splitting the disease string into words, look for the
  closest CUI concept, where distance is defined via the damerau-levelshtein string distance.
  """
  if disease in str_to_cui: 
    
    return [str_to_cui[disease]]

  # when the disease is not found in the str_to_cui dictionary
  words = disease.split(" ")
  words = [w.split("-") for w in words]
  words = flatten_lol(words)

  cuis = []

  for i,w in enumerate(words):

      if w in str_to_cui:

          cuis.append(str_to_cui[w])
  
  if len(cuis) == 0:

    print(f"There was no CUI match for the words in disease {disease}.\n")

    distances = np.zeros(len(str_to_cui), dtype=float)

    # compute string distance to all other CUI concepts
    for i,s in enumerate(str_to_cui.keys()):

      distances[i] = jellyfish.damerau_levenshtein_distance(disease, str(s))

    print(f"The closest string we could find was {list(str_to_cui.keys())[np.argmin(distances)]}. \n")

    cuis = [str_to_cui[list(str_to_cui.keys())[np.argmin(distances)]]]
    
  
  return cuis

In [ ]:
# match all diseases to a CUI
diseases_to_cuis = {}

for d in tqdm(diseases):

  diseases_to_cuis[d] = d_to_cuis(d)

  0%|          | 0/1586 [00:00<?, ?it/s]

There was no CUI match for the words in disease chronic obstructive.



  1%|          | 10/1586 [00:24<1:05:09,  2.48s/it]

The closest string we could find was chronic obstruction. 

There was no CUI match for the words in disease rheumatoid.



  1%|          | 11/1586 [00:41<1:50:41,  4.22s/it]

...

The closest string we could find was health. 

There was no CUI match for the words in disease ankylosing.



 98%|█████████▊| 1562/1586 [05:52<00:07,  3.06it/s]

The closest string we could find was ankylosis. 

There was no CUI match for the words in disease myelocytic.



100%|██████████| 1586/1586 [06:08<00:00,  4.31it/s]

The closest string we could find was myelocytes. 



In [ ]:
# cui to embedding
cui_emb = pd.read_csv(deep_learning_dir + cui_path + "/cui2vec_pretrained.csv")
cui_emb.set_index("Unnamed: 0", inplace=True)

embeddings = {}

for index, row in cui_emb.iterrows():
    embeddings[index] = np.array(row)

In [ ]:
# compute embedding of every disease with at least one existing corresponding CUI
# Note: a disease can have multiple corresponding CUI's

d_emb = {}

for d, cuis in diseases_to_cuis.items():
  
  embs = [embeddings[cui] for cui in cuis if cui in embeddings]

  if len(embs) != 0:

    d_emb[d] = np.mean(embs, axis=0)

In [ ]:
def embedding_row(diseases):
  """
  Embed a list of diseases. Will correspond to the diseases associated to a single clinical trial
  """

  d_embs = [d_emb[d] for d in diseases if d in d_emb]

  if len(d_embs) == 0:
    # 500 for embedding
    return np.zeros(500, dtype=float)

  else:
    # 500 for average disease embedding
    return np.mean(d_embs, axis=0)

In [ ]:
# key: nctid of clinical trial, value: disease embedding

train_embeddings = [embedding_row(diseases) for diseases in diseases_train]
train_n_diseases = [len(d) for d in diseases_train]
train_d_embeddings = dict(zip(train["nctid"], train_embeddings))
train_n_diseases = dict(zip(train["nctid"], train_n_diseases))

val_embeddings = [embedding_row(diseases) for diseases in diseases_val]
val_n_diseases = [len(d) for d in diseases_val]
val_d_embeddings = dict(zip(val["nctid"], val_embeddings))
val_n_diseases = dict(zip(val["nctid"], val_n_diseases))


test_embeddings = [embedding_row(diseases) for diseases in diseases_test]
test_n_diseases = [len(d) for d in diseases_test]
test_d_embeddings = dict(zip(test["nctid"], test_embeddings))
test_n_diseases = dict(zip(test["nctid"], test_n_diseases))

# combinded dictionary
final_d_embeddings = {**train_d_embeddings, **val_d_embeddings, **test_d_embeddings}
final_n_diseases = {**train_n_diseases, **val_n_diseases, **test_n_diseases}

In [ ]:
with open("nctid2diseases.pkl", 'wb') as handle:
    pickle.dump(final_d_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("nctid2n_diseases.pkl", 'wb') as handle:
    pickle.dump(final_n_diseases, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!mv nctid2diseases.pkl "/content/gdrive/My Drive/BMI 707 Project/embeddings/"
!mv nctid2n_diseases.pkl "/content/gdrive/My Drive/BMI 707 Project/embeddings/"